In [ ]:
import sys, os
#default path for current release 
sys.path.append("C:\\Program Files\\Lumerical\\v202\\api\\python\\") 
# # sys.path.append(os.path.dirname(__file__)) #Current directory

import ctypes
interopapi_path = r"C:\Program Files\Lumerical\v202\api\python\interopapi.dll"
ctypes.CDLL(interopapi_path)

import lumapi
import lumapi
import importlib.util
import numpy as np
import scipy.io as sio

In [ ]:

L1 = 35e-6  # Outer U height
L2 = 35e-6  # Outer U width
w = 2.5e-6  # Width of the arms
d = 4e-6    # Gap between the two U-shapes

# parameters for the inner U
L3 = L1 - 2 * d
L4 = L2 - 2 * d

period = 40e-6          # Periodicity
thickness_Al = 2e-6       # Bottom 
thickness_SiO2 = 6e-6     # Middle 

t_Ag =100e-9


thickness_Ag_values = np.linspace(100e-9, 700e-9, 7) # Sweep


current_directory = os.getcwd()
new_folder_name = "Ag_thickness_sweep_results"
new_folder_path = os.path.join(current_directory, new_folder_name)
os.makedirs(new_folder_path, exist_ok=True)
print(f"Results will be saved in: {new_folder_path}")


# DRU structure----------------------------------

def add_dru_structure(fdtd, z_center, z_span):
    """Adds the double U-shaped (DRU) structure using multiple rectangles."""
  
    fdtd.addrect(name="outer_U_base",
                 x=0, x_span=L2,
                 y=(-L1/2) + (w/2), y_span=w,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")
    
    fdtd.addrect(name="outer_U_left_arm",
                 x=(-L2/2) + (w/2), x_span=w,
                 y=0, y_span=L1,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")
   
    fdtd.addrect(name="outer_U_right_arm",
                 x=(L2/2) - (w/2), x_span=w,
                 y=0, y_span=L1,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")

   
    fdtd.addrect(name="inner_U_base",
                 x=0, x_span=L4-2*d,
                 y=(-L1/2) + (w/2) + 2*d, y_span=w,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")
    
    fdtd.addrect(name="inner_U_left_arm",
                 x=(-L2/2) + (w/2) + 2*d, x_span=w,
                 y=0, y_span=L3-2*d,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")
   
    fdtd.addrect(name="inner_U_right_arm",
                 x=(L2/2) - (w/2) - 2*d, x_span=w,
                 y=0, y_span=L3-2*d,
                 z=z_center, z_span=z_span,
                 material="Au (Gold) - Palik")


sweep_results = []
count = 0





for t_Ag in thickness_Ag_values:
    
    with lumapi.FDTD() as fdtd:
        count += 1
        t_Ag_nm = t_Ag * 1e9
        print(f"Running simulation {count}/{len(thickness_Ag_values)}: Ag thickness = {t_Ag_nm:.0f} nm")

     
        fdtd.switchtolayout()


        z0 = 0  # Reference: bottom of Al layer
        z_Al_c = z0 + thickness_Al / 2
        z_SiO2_c = z0 + thickness_Al + thickness_SiO2 / 2
        z_Ag_c = z0 + thickness_Al + thickness_SiO2 + t_Ag / 2

        z_buffer = 4e-6 # Buffer space above and below the structure
        z_min = z0 - z_buffer
        z_max = z_Ag_c + t_Ag / 2 + z_buffer

        fdtd.addfdtd()
        fdtd.setnamed("FDTD", {
            "dimension": "3D",
            "x span": period, "y span": period,
            "z min": z_min, "z max": z_max,
            "x min bc": "Periodic", "x max bc": "Periodic",
            "y min bc": "Periodic", "y max bc": "Periodic",
            "z min bc": "PML", "z max bc": "PML",
            "mesh accuracy": 3
            
        })

    
        fdtd.addrect(name="Al_bottom",
                        x_span=period, y_span=period,
                        z=z_Al_c, z_span=thickness_Al,
                        material="Al (Aluminium) - Palik")

 
        fdtd.addrect(name="SiO2_mid",
                        x_span=period, y_span=period,
                        z=z_SiO2_c, z_span=thickness_SiO2,
                        material="SiO2 (Glass) - Palik")

        
        add_dru_structure(fdtd, z_center=z_Ag_c, z_span=t_Ag) # Ag layer with sweep thickness
        # # Mesh override for better accuracy in thin layers
        fdtd.addmesh(name="mesh_stack",
                        x_span=period, y_span=period,
                        z_min=z0 - 0.5e-6, z_max=z_Ag_c + t_Ag/2 + 0.5e-6,
                        override_x_mesh=True, dx=0.5e-6,
                        override_y_mesh=True, dy=0.5e-6,
                        override_z_mesh=True, dz=0.05e-6)

        # Broadband plane wave source (0.1-2 THz)
        fdtd.addplane(name="source",
                        injection_axis="z", direction="backward",
                        x_span=period, y_span=period,
                        z=z_Ag_c + t_Ag/2 + 1e-6,
                        wavelength_start=15e-5, 
                        wavelength_stop=3e-3)    # 0.1 to 2Thz

        # Transmission monitor above the source
        fdtd.addpower(name="T_monitor",
                        monitor_type="2D Z-Normal",
                        x_span=period, y_span=period,
                        z=z_Ag_c + t_Ag/2 + 2e-6)


        filename = f"metastructure_Ag_{t_Ag_nm:.2f}nm.fsp"
        file_path = os.path.join(new_folder_path, filename)
        fdtd.save(file_path)

        # Run the FDTD simulation
        fdtd.run()

        T_result = fdtd.getresult("T_monitor", "T")
        
        
        sweep_results.append({
            'Ag_thickness_nm': t_Ag_nm,
            'T': T_result['T'],
            'lambda': T_result['lambda'],
            'f': T_result['f']
        })


summary = []
for run in sweep_results:
    thickness_nm = float(run['Ag_thickness_nm'])
    lam = np.asarray(run['lambda']).ravel()
    T   = np.asarray(run['T']).ravel()

    
    peak_T = float(np.max(T))

    summary.append({
        "Ag_thickness_nm": thickness_nm,
        "peak_T": peak_T,
    })


thicknesses = np.array([s["Ag_thickness_nm"] for s in summary])
peak_T_vals = np.array([s["peak_T"] for s in summary])


i_peak = int(np.argmax(peak_T_vals))



mat_filename = os.path.join(new_folder_path, "sweep_summary.mat")

save_dict = {f"run_{i}": result for i, result in enumerate(sweep_results)}
save_dict['Ag_thickness_values_nm'] = thicknesses
save_dict['summary'] = {
    'Ag_thickness_nm': thicknesses,
    'peak_T': peak_T_vals,
    'best_peak_T_wavelength': np.array([thicknesses[i_peak]]),
    'best_peak_T_value': np.array([peak_T_vals[i_peak]]),
}

sio.savemat(mat_filename, save_dict)
print(f"All results saved to {mat_filename}")


